<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')

<ipython-input-11-44878571b353>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')


In [18]:
from google.colab import files
uploaded = files.upload()

Saving preprocessing.py to preprocessing.py


In [7]:
!pip install convertdate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.8 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732000 sha256=b0143dfa7e8d9fc6b33fbbc5362ee00f82a0a38b01f00f05e1db340e7d9da6a3
  Stored in directory: /root/.cache/pip/wheels/8f/bd/f9/5c4c39b529e0322b08979e1c465e203218bc2cca75d20f7df5
Successfully built pymeeus


In [19]:
# אם הקובץ נקרא preprocessing.py
import preprocessing

# טען את הפונקציה
from preprocessing import prepare_data


In [20]:
df_ready = prepare_data(df)

KeyError: "Column(s) ['כמות'] do not exist"

In [10]:
df.columns

Index(['Date', 'Month', 'סוג פריט', 'תאור סוג', 'מס' פריט', 'תאור פריט',
       'הזמנה', 'כמות', 'סכום', 'הנחה לפריט', 'אחרי הנחה', 'Is_original'],
      dtype='object')

In [21]:
!grep -n "כמות" preprocessing.py


12:    df = df[df["כמות"] >= 0]
31:        'תאור פריט','הזמנה','כמות','סכום',
126:        base_data = row.drop(["כמות בפועל", "תאור פריט", "כמות"], errors="ignore").to_dict()
128:        quantity_dict = row.get("כמות בפועל", {})
129:        default_qty = row["כמות"]
361:    df["כמות בפועל"] = df["תאור פריט"].apply(extract_quantity_by_keywords)


In [22]:
!grep -R "כמות" preprocessing.py
!grep -R "agg" preprocessing.py


    df = df[df["כמות"] >= 0]
        'תאור פריט','הזמנה','כמות','סכום',
        base_data = row.drop(["כמות בפועל", "תאור פריט", "כמות"], errors="ignore").to_dict()
        quantity_dict = row.get("כמות בפועל", {})
        default_qty = row["כמות"]
    df["כמות בפועל"] = df["תאור פריט"].apply(extract_quantity_by_keywords)
    agg_features = df.groupby("clean_desc_encoded").agg({
    agg_features.columns = ["avg_quantity_all_time", "std_quantity_all_time", "num_days_sold"]
    agg_features = agg_features.reset_index()
    agg_features["popularity_score"] = agg_features["avg_quantity_all_time"]
    df = df.merge(agg_features, on="clean_desc_encoded", how="left")


In [23]:
!sed -n '350,380p' preprocessing.py



def sort_by_date(df):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    
    return df.sort_values("Date")

def prepare_data(df):
    df = drop(df)
    df = convert_date(df)
    df = split_category(df)
    df["כמות בפועל"] = df["תאור פריט"].apply(extract_quantity_by_keywords)
    df = split_rows_by_category_and_quantity(df)
    df = add_holiday_features(df)
    df = encode_features(df)
    df = add_time_features(df)
    df = add_product_features(df)
    df = encode_holiday_and_portion(df)
    df = clean_final_columns(df)
    df = sort_by_date(df)
    
    return df



  

